In [1]:
import numpy as np
import pandas as pd
import os 

import tensorflow as tf
from sklearn.model_selection import train_test_split

from DeepModel import DeepModel
from Scaling import Scaling

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import shuffle

#tensorboard --logdir=logs/
#!{sys.executable} -m pip install tensorflow-addons 

In [2]:
model_folder = '../model'

checkpoint_path = model_folder + "/TerrAI.ckpt"

In [3]:
if not os.path.exists(model_folder):
    os.mkdir(model_folder)

In [4]:
scaler = Scaling()

## Ensure available GPU utilized correctly

In [5]:
physical_devices = tf.config.list_physical_devices("GPU")

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Load training data

In [6]:
data = pd.read_csv('../Dataset/Train.csv').dropna()
data.shape

(10864, 227)

In [7]:
# Select main columns to be used in training
X = scaler.reshape_and_scale_X(data)
y = np.expand_dims(data.Label.to_numpy(dtype=np.float64), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

In [8]:
X.shape, y.shape

((10864, 9, 5, 5), (10864, 1))

In [9]:
input_size = X_train.shape[1:]
input_size

(9, 5, 5)

## Calculate the output class distribution percentage
Used in order to ensure class weight adjusted properly

In [10]:
zero_state_count = len(y[y == 0])
total_label_count = len(y)
zero_state_count, total_label_count

(8148, 10864)

In [11]:
zero_weight = 1
one_weight = 1 / total_label_count * zero_state_count + 1

class_weight = {0: zero_weight, 1: one_weight}
class_weight

{0: 1, 1: 1.75}

## Dataset generator

In [12]:
batch_size = 512

In [13]:
def fire(X, y, batch_size=64):
    return_index = 0
    max_index = len(y) // batch_size
    X, y = shuffle(X, y)
    
    for i in range(1, max_index):
        yield X[batch_size * (i-1):batch_size * i], y[batch_size * (i-1):batch_size * i]

In [14]:
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(X=X_train, y=y_train, batch_size=batch_size),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1]))

test_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(X=X_test, y=y_test, batch_size=batch_size),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1]))

In [15]:
fire_iter = iter(fire(X=X_test, y=y_test))
a, b = next(fire_iter)
a.shape, b.shape

((64, 9, 5, 5), (64, 1))

In [16]:
model = DeepModel(checkpoint_path, 1, input_size)

In [ ]:
model.train(train_dataset, test_dataset, class_weight=class_weight, epochs=300, batch_size=batch_size)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 5, 32)          1472      
_________________________________________________________________
activation (Activation)      (None, 9, 5, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 9, 5, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 5, 8)           2312      
_________________________________________________________________
activation_1 (Activation)    (None, 9, 5, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 5, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 360)               0